# Init

In [ ]:
# To work with files
import os

# To use DataFrames
import pandas as pd

# np.where, np.size 
import numpy as np

# To parse time
from datetime import datetime, timedelta 

# Used in funct def f_comp_angle(row):
import math

# To plot graphs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib nbagg

# To allow more outputs in Jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# To disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

# Path to left & right wrist Actigraphy, and Polysomnography data
path = "C:\\Users\\sigmu\\Desktop\\dataset\\"
path_ACG = path + "Aktigrafie\\"
path_PSG = path + "Polysomnografie\\"
files_ACG = os.listdir(path_ACG)
files_PSG = os.listdir(path_PSG)

#good accuracy parameters: first_threshold = 10 minutes, time_window = 3 minutes, angle = 10 (divided by 5 ... 5 s resampling)
first_threshold = (10 * 60) / 5
time_window = (3 * 60) / 5
angle = 10

# df to write result to
df_stats = pd.DataFrame(columns=['Participant', 
                                 'TIB', 'SOL', 'TST', 'WASO', 'NA>5', 'SFI', 'SWR', 'SE%',
                                 'sensitivity', 'specificity', 'accuracy', 'MCC'])

df_PSG_stats = pd.DataFrame(columns=['Participant', 
                                     'TIB', 'SOL', 'TST', 'WASO', 'NA>5', 'SFI', 'SWR', 'SE%'])

# Functions

In [ ]:
# function to cut start of the DataFrames to match in time
def f_cut_start(df, df_PSG):    
    # Match start
    ## If ACG starts earlier than PSG -> cut start of ACG
    if(df['time stamp'][0] < df_PSG['Time [hh:mm:ss]'][0]):          
    # Find df index where time matches    
        try:
            idx_start = df[(df['time stamp'].dt.day == df_PSG['Time [hh:mm:ss]'].dt.day[0]) &
                           (df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[0]) &
                           (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[0]) &
                           (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[0])].index[0]
            # Drop df from 0 to idx_start            
            df.drop(df.index[0:(idx_start)], inplace=True)
        except:
            print("ACG has no identical time stamp start with PSG: " + participant+'_'+arm)
    ## Else cut start of PSG        
    else:
        # Find df_PSG index where time matches   
        try:
            idx_start = df_PSG[(df['time stamp'].dt.day == df_PSG['Time [hh:mm:ss]'].dt.day[0]) &
                               (df['time stamp'].dt.hour[0] == df_PSG['Time [hh:mm:ss]'].dt.hour) & 
                               (df['time stamp'].dt.minute[0] == df_PSG['Time [hh:mm:ss]'].dt.minute) &
                               (df['time stamp'].dt.second[0] == df_PSG['Time [hh:mm:ss]'].dt.second)].index[0]
            # Drop df_PSG from 0 to idx_start            
            df_PSG.drop(df_PSG.index[0:(idx_start)], inplace=True) 
        except:
            print("PSG has no identical time stamp with ACG: " + participant+'_'+arm)

    return

# function to cut end of the DataFrames to match in time
def f_cut_end(df, df_PSG): 
    df.reset_index(inplace=True, drop=True)
    df_PSG.reset_index(inplace=True, drop=True)
    df_PSG_len = len(df_PSG.index)-1
    df_len = len(df.index)-1

    # Match end
    ## If df ends earlier than df_PSG -> cut end of df_PSG
    if(df['time stamp'][df_len] < df_PSG['Time [hh:mm:ss]'][df_PSG_len]):
        try:
            idx_end = df_PSG[(df['time stamp'].dt.hour[df_len] == df_PSG['Time [hh:mm:ss]'].dt.hour) &
                             (df['time stamp'].dt.minute[df_len]+1 == df_PSG['Time [hh:mm:ss]'].dt.minute)].index[0]  # + 1 min  
            # Drop df_PSG from df end to end
            df_PSG.drop(df_PSG.index[(idx_end):(len(df_PSG.index))], inplace=True)
        except:
            print("PSG has no identical time stamp end with ACG: " + participant+'_'+arm)
    ## Else cut end of df
    else:
        try:
            idx_end = df[(df['time stamp'].dt.day == (df_PSG['Time [hh:mm:ss]'].dt.day[df_PSG_len])) &
                         (df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[df_PSG_len]) &
                         (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[df_PSG_len]) &
                         (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[df_PSG_len])].index[0]
            # Drop df from df_PSG end to end
            df.drop(df.index[(idx_end):(len(df.index))], inplace=True)
        except:
            print("ACG has no identical time stamp end with PSG: " + participant+'_'+arm)
        
    return
        
# ----------------------------------------------------------------------------------------------------------- 

# Function to compute angle according to van Hees method
def f_comp_angle(row):
    return math.degrees(math.atan(row['z axis [g]'] / ((row['x axis [g]']**2 + row['y axis [g]']**2)**0.5)))

# -----------------------------------------------------------------------------------------------------------  

# Function to decide sleep & wake epochs, first sleep time and sleep end time, if the accelerometer has been worn, 
# sleep fragmentation and number of awakenings > 5
# parameters: window of inactivity to decide SO, window to decide sleep after SO (both in seconds / 5 ... 5s resampling), 
# angle threshold and ACG dataframe

def f_inactiv(first_threshold, time_window, angle, df):
    sleep_onset = sleep_end = sleep_fragmentation = counter = non_wear_counter = five_min = NA_5 = 0
    first_sleep = woke_up = frag = False
    result = []
        
    for index, value in df['abs_angle_change'].items():
        counter += 1

        # Non-wear possible solution: if angle change is 0 for at least an hour
        if (value == 0):
            non_wear_counter += 1
            if(non_wear_counter == 720):
                df['ACG_state'] = "N"
                break
        else:
            non_wear_counter = 0
                
        # Angle change > angle -> woke up
        if (value > angle):
            counter = 0                                 
        # After first sleep  
        elif (counter > time_window) & first_sleep:
            # Write S to state
            df.loc[index, 'ACG_state'] = "S"            
            five_min = 0
            frag = False
            woke_up = True
            # Sleep End (end of sleep period)
            sleep_end = index
        # First sleep 
        elif (counter > first_threshold) & ~first_sleep:
            # Write S to state
            df.loc[index, 'ACG_state'] = "S"
            first_sleep = True
            # Sleep Onset (start of sleep period)
            sleep_onset = index  
        
        # after SO: if woken up, add to Sleep Fragmentation count
        if ((df.loc[index, 'ACG_state'] == 'W') & first_sleep):                                          
            if(frag == False):
                sleep_fragmentation += 1
                frag = True  
            
            five_min += 1
            # if greater than 5 minutes, add to NA>5
            if ((five_min > (5*60/5)) & woke_up):
                NA_5 += 1  
                woke_up = False
            
    result.append(sleep_onset)
    result.append(sleep_end)
    result.append(sleep_fragmentation)
    result.append(NA_5)
    return result
 
# -----------------------------------------------------------------------------------------------------------

# Function to compute sleep parameters
# parameters: ACG dataframe, result dataframe to write statistics in
def f_stats(df, result):      
    
    # If non-wear
    if(df['ACG_state'][0] == 'N'):
        print("Accelerometr is not being used in recording: " + participant+'_'+arm)
        return
    
    # TIB is presumed to be whole recording after f_cut_start, f_cut_end, in minutes
    TIB = (df.index[len(df)-1] - df.index[0]) / np.timedelta64(1,'m')
    
    # Sleep Onset and last Sleep from f_inactiv func (as timestamp -> str)
    try:
        sleep_onset = str(inactivity[0].time())    
        sleep_end = str(inactivity[1].time())
        last = df['ACG_state'][len(df)-1]
        sleep_end_greater = df.index[len(df)-1] < inactivity[1]
        # If last item in dataframe is sleep and it is less than sleep_end (sleep_end was created with 5s resampling - could be after last item)
        if ((last == 'S') & sleep_end_greater):
            sleep_end = str(df.index[len(df)-1].time())
        # Sleep Onset Latency: sleep_onset - start of recording
        SOL = inactivity[0] - df_PSG['Time [hh:mm:ss]'][0]
        SOL = round(SOL.seconds / 60, 2)
    except:
        sleep_onset = 'NaN'
        sleep_end = 'NaN'
        SOL = 'NaN'
        print("SOL is not registered in recording: " + participant+'_'+arm)    
    
    # TST is the duration in minutes of all sleep epochs
    TST = (len(df[df['ACG_state']=='S'].index.value_counts()) * 30 ) / 60

    # WASO is the duration in minutes of all wake epochs between SOL time and sleep end
    WASO = ( len(df[(df['ACG_state']=='W')&(df.index > inactivity[0])]) * 30) / 60
    
    # NA>5 is number of awakenings after SO with duration greater than 5 minutes
    try:
        NA_5 = inactivity[3]
    except:
        NA_5 = 'NaN'

    # Sleep Fragmentation is number of intervals scored as "W" (after sleep onset) relative to the total sleep time in hours
    try:
        SFI = round(inactivity[2] / (TST/60), 2)
    except:
        SFI = 'NaN'
        
    # Sleep Wake Ratio
    try:
        SWR = round(TST / WASO, 2)
    except:
        SWR = 'NaN'
    
    # Sleep Eficiency
    try:
        SE = round((TST / TIB)*100, 2)
    except:
        SE = 'NaN'
    
    #sensitivity (actigraphy = sleep when PSG = sleep), 
    #specificity (actigraphy = wake when PSG = wake), 
    #and accuracy (total proportion correct)
    # Put all states where: into an array
    TP = np.where((df['ACG_state'] == 'S') & (df['PSG_state'] == 'S'))
    FP = np.where((df['ACG_state'] == 'S') & (df['PSG_state'] == 'W'))
    TN = np.where((df['ACG_state'] == 'W') & (df['PSG_state'] == 'W'))
    FN = np.where((df['ACG_state'] == 'W') & (df['PSG_state'] == 'S'))

    # Return array size
    TP_cnt = np.size(TP)
    FP_cnt = np.size(FP)
    TN_cnt = np.size(TN)
    FN_cnt = np.size(FN)

    # Calculate sensitivity, specificity and accuracy
    sens = round((TP_cnt / (TP_cnt + FN_cnt) )*100, 2)
    spec = round((TN_cnt / (TN_cnt + FP_cnt) )*100, 2)
    acc = round(((TP_cnt + TN_cnt) / (TP_cnt + TN_cnt + FP_cnt + FN_cnt) )*100, 2)
    
    # Matthews correlation coefficient: range -1 to 1, perfect is 1, always missclassifies when -1 and 0 is a coin flip
    try:
        MCC = round((TP_cnt*TN_cnt-FP_cnt*FN_cnt)/math.sqrt((TP_cnt+FP_cnt)*(TP_cnt+FN_cnt)*(TN_cnt+FP_cnt)*(TN_cnt+FN_cnt)), 2)
    except ZeroDivisionError:
        MCC = 'NaN'

    # Add to stats dataframe    
    result = result.append({'Participant': participant+'_'+arm,
                            'TIB': TIB, 'SOL': SOL, 'TST': TST, 'WASO': WASO, 'NA>5': NA_5, 'SFI': SFI, 'SWR': SWR, 'SE%': SE,
                            'sensitivity': sens, 'specificity': spec, 'accuracy': acc, 'MCC': MCC}, 
                           ignore_index=True)
    
    return result

def f_PSG_inactiv(df):
    SO_index = five_min = NA_5 = s_fragment = 0
    woke_up = frag = True
    result = []    

    SO_index = df[(df['PSG_state']=='S')].index[0]
    
    # NA>5 & SFI for PSG
    for index, value in df['PSG_state'].items():    
        if((index > SO_index) & (value == 'W')):              
            five_min += 1

            if((five_min > 10) & woke_up):
                NA_5 += 1
                woke_up = False

            if(frag):
                s_fragment += 1
                frag = False
        else:
            five_min = 0
            woke_up = frag = True
                    
    result.append(SO_index)
    result.append(NA_5)
    result.append(s_fragment)
    return result

def f_PSG_stats(df, result):  
    
    # TIB is presumed to be whole recording after f_cut_start, f_cut_end, in minutes
    TIB = (df_PSG['Time [hh:mm:ss]'][len(df_PSG)-1] - df_PSG['Time [hh:mm:ss]'][0]) / np.timedelta64(1,'m')
    
    # timestamp first instance of 'S' minus timestamp start of df, in minutes
    # in df is PSG_state column with PSG states into binary 'W' and 'S'
    SO_index = PSG_inactivity[0]
    SOL = (SO_index - df.index[0]) / np.timedelta64(1,'m')

    
    # TST is the duration in minutes of all sleep epochs
    TST = (len(df[df['PSG_state']=='S'].index.value_counts()) * 30 ) / 60

    # WASO is the duration in minutes of all wake epochs between SOL time and sleep end
    WASO = ( len(df[(df['PSG_state']=='W')&(df.index > SO_index)]) * 30) / 60
    
    # NA>5 is number of awakenings after SO with duration greater than 5 minutes
    try:
        NA_5 = PSG_inactivity[1]
    except:
        NA_5 = 'NaN'

    # Sleep Wake Ratio
    try:
        SWR = round(TST / WASO, 2)
    except:
        SWR = 'NaN'
    
    # Sleep Eficiency
    try:
        SE = round((TST / TIB)*100, 2)
    except:
        SE = 'NaN'
    
    # Sleep Fragmentation is number of intervals (duration until another "S") scored as "W" (after sleep onset) relative to the total sleep time in hours
    try:
        SFI = round(PSG_inactivity[2] / (TST/60), 2)
    except:
        SFI = 'NaN'
    
    # Add to stats dataframe    
    result = result.append({'Participant': participant,
                            'TIB': TIB, 'SOL': SOL, 'TST': TST, 'WASO': WASO, 'NA>5': NA_5, 'SFI': SFI, 'SWR': SWR, 'SE%': SE}, 
                           ignore_index=True)
    
    return result

# Iterate through files

In [ ]:
%%time

# go through PSG files: eg. mecsleep01_psg.txt (take 1 file and continue)
for file_name_PSG in files_PSG:
    print(file_name_PSG)
    # split on _, take only mecsleep01 and put to uppercase
    participant = file_name_PSG.split("_")[0].upper() 
    # go through ACG files: take left and right arm rec (2 files for 1 PSG file)
    for file_name_ACG in files_ACG: 
        arm = file_name_ACG.split("_")[1]
        if(file_name_ACG.startswith(participant+"_"+arm)):
            print(file_name_ACG)
            df = pd.read_csv(path_ACG+file_name_ACG, names=['time stamp', 'x axis [g]', 'y axis [g]', 'z axis [g]', 
                                                            'light level [lux]', 'button [1/0]','temperature [°C]'],
                             skiprows=100,
                             # might be slightly faster:
                            infer_datetime_format=True, memory_map=True)
            df['time stamp'] = pd.to_datetime(df['time stamp'], format='%Y-%m-%d %H:%M:%S:%f')
            # drop not used columns from ACG 
            df.drop(columns=['light level [lux]', 'button [1/0]', 'temperature [°C]'], inplace=True, axis=1)
            
            # 1st PSG read -> get recording date
            df_PSG = pd.read_csv(path_PSG + file_name_PSG,
                                infer_datetime_format=True, memory_map=True)
            PSG_date = df_PSG["RemLogic Event Export"][2].split("\t")[1]
            # 2nd PSG read -> parse to datetime
            df_PSG = pd.read_csv(path_PSG + file_name_PSG, sep='\t', skiprows=17,
                                infer_datetime_format=True, memory_map=True)
            df_PSG['Time [hh:mm:ss]'] = PSG_date + " " + df_PSG['Time [hh:mm:ss]']
            df_PSG['Time [hh:mm:ss]'] = pd.to_datetime(df_PSG['Time [hh:mm:ss]'], format='%d/%m/%Y %H:%M:%S')
            PSG_date = pd.to_datetime(PSG_date, format = '%d/%m/%Y')                       
            #add a day if PSG crosses 00:00
            midnight = df_PSG[(df_PSG['Time [hh:mm:ss]'].dt.hour == 0) &
                                  (df_PSG['Time [hh:mm:ss]'].dt.minute == 0)]
            mid_idx = midnight.index[0]
            if(~midnight.empty & mid_idx != 0):
                for index, value in df_PSG['Time [hh:mm:ss]'].items():
                    if(index >= mid_idx):
                        df_PSG['Time [hh:mm:ss]'][index] += pd.to_timedelta(1,unit='d')
            
            #-----------------------------------------------------------------------------------------------
            
            # Drop values so that the ACG and PSG recording is starting and ending at the same time
            f_cut_start(df, df_PSG)
            f_cut_end(df, df_PSG)

            #-----------------------------------------------------------------------------------------------

            # Set origin from PSG to start resampling
            orig = (df_PSG['Time [hh:mm:ss]'].dt.hour[0]*3600 + 
                    (df_PSG['Time [hh:mm:ss]'].dt.minute[0]*60) +
                    (df_PSG['Time [hh:mm:ss]'].dt.second[0]))
            orig  = pd.Timestamp(orig, unit='s')

            # Resample by 5 second epoch and compute median of x,y,z
            df = df.resample('5S', on='time stamp', kind='timestamp', origin=orig).median()#.round(decimals=4)

            #-----------------------------------------------------------------------------------------------  

            # Apply func f_comp_angle
            df['angle'] = df.apply(f_comp_angle, axis=1)#.round(decimals=4)

            # Return absolute difference in angle per column
            df['abs_angle_change'] = df['angle'].diff().abs()#.round(decimals=4)
            
            # New column with all "W" values
            df['ACG_state'] = "W"         

            #-----------------------------------------------------------------------------------------------    

            # Decide inactivity based on the angle            
            # returns list of timestamps for SE, SO
            inactivity = f_inactiv(first_threshold, time_window, angle, df)
            
            if(df['ACG_state'][0] != 'N'):

                #-----------------------------------------------------------------------------------------------

                # Resample ACG to have same number of columns as PSG (30s epochs)
                df = df.resample('30S', origin=orig).interpolate()

                # If PSG has extra value at the end
                if(len(df_PSG) > len(df)):
                    df_PSG.drop(df_PSG.index[len(df_PSG)-1], inplace=True)

                # Overwrite State PSG to bi-state 
                df['PSG_state'] = np.where((df_PSG['Sleep Stage'] == 'N1')|(df_PSG['Sleep Stage'] == 'N2')|
                                                   (df_PSG['Sleep Stage'] == 'N3')|(df_PSG['Sleep Stage'] == 'R'), 'S', 'W')

                # Create boolean column and compare
                df['match'] = np.where(df['ACG_state'] == df['PSG_state'], True, False)
                
                PSG_inactivity = f_PSG_inactiv(df)

                #-----------------------------------------------------------------------------------------------

                # Compute statistics
                df_stats = f_stats(df, df_stats)                                          
    df_PSG_stats = f_PSG_stats(df, df_PSG_stats)  
                #-----------------------------------------------------------------------------------------------          

# Write to file

In [ ]:
returns_path_csv = path + "Results\\" + "df.csv"                
df.to_csv(returns_path_csv)   

In [ ]:
returns_path_csv = path + "Results\\" + "final_STATS.csv"                
df_stats.to_csv(returns_path_csv)       

In [ ]:
returns_path_csv = path + "Results\\" + "final_PSG_STATS.csv"                
df_PSG_stats.to_csv(returns_path_csv)  

# Read from file

In [ ]:
read_path_csv = path + "Results\\"+ "STATS.csv"
stats = pd.read_csv(read_path_csv)
stats